In [ ]:
class repo_trading:
    
    '''
    Class of function to automate repo trading process.
    
    '''
    
    def close_repo(collateral, financing):
        
        '''
        Return a dataframe with all the funding trades to close and reopen to cover short positions.
        
        collateral (dataframe): df from aim_manipulation.collateral_positions()
        financing (dataframe): df from aim_manipulation.financing_trade_blotter()
        
        '''
                      
        def first_short_position(collateral):
            
        # Create a matrix with the collateral postions for a 5-day horizon and find the first day a balance becomes negative 
        # for each security
            
            df_matrix = collateral[["T+0", "T+1", "T+2", "T+3", "T+4"]]
            
            negative_indices = []
            
            for index, row in df_matrix.iterrows():
                
                for col in df_matrix.columns:
                    
                    if row[col] < 0:
                        
                        negative_indices.append((index, col))
                        
                        break
                        
            return negative_indices
        
        short_positions = first_short_position(collateral) 
        
        df_short = collateral.loc[[pos[0] for pos in short_positions], # extract index from the rows that have a negative balance
                                
                                ["Account", "Curr", "Security", "ISIN"]].assign( # specify columns to select
            
            Position = [collateral.loc[pos[0], pos[1]] for pos in short_positions], 
            
            Settlement = [pos[1] for pos in short_positions]).sort_values(by = ["Account", "Security"],
                                                                         
                                                                         ascending = [True, True]).reset_index(drop = True)
        
        instructions = []
        
        for i, short in df_short.iterrows():
            
            df_temp = financing[(financing["Identifier"] == short["ISIN"]) & 
                               
                               (financing["Account"] == short["Account"])].sort_values(by = ["Rate", "Quantity"], 
                                                                                          
                                                                                          ascending = [False, True]).reset_index(drop = True)
            
            position_to_cover = short["Position"]
            
            for j, repo in df_temp.iterrows():
                
                if position_to_cover > 0:
                    
                    break
                    
                if  position_to_cover + repo["Quantity"] > 0:
                
                    to_reopen = position_to_cover + repo["Quantity"]
                    
                else:
                    
                    to_reopen = 0
                    
                instructions.append({
                    
                    "Account" : repo["Account"],
                    
                    "Collateral" : repo["Collateral"],
                    
                    "Identifier" : repo["Identifier"],
                    
                    "Close" : repo["Quantity"],
                    
                    "Open" : to_reopen,
                    
                    "Currency" : repo["Curr"],
                    
                    "Settlement": short["Settlement"],
                    
                    "Rate" : repo["Rate"],
                    
                    "Broker" : repo["Broker"]                    
                    
                })
                
                position_to_cover += repo["Quantity"]
                
        df_instructions = pd.DataFrame(instructions)
            
        return df_instructions